In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam

In [25]:
# Load the air quality dataset
df = pd.read_csv('kurla_cleaned.csv')

In [26]:
# Convert the date column to a datetime object
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y %H:%M')

In [27]:
# Set the date column as the index
df.set_index('date', inplace=True)

In [28]:
# Select the columns to be used as input features
features = ['PM2.5', 'PM10', 'NO', 'NO2', 'SO2', 'CO', 'Ozone', 'RH', 'WS', 'WD', 'AT']


In [29]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(df[features], test_size=0.2, shuffle=False)

In [30]:
# Normalize the training data
train_mean = train_data.mean()
train_std = train_data.std()
train_data = (train_data - train_mean) / train_std

In [31]:
# Define the input and output sequences
input_seq_len = 24 * 7 # 7 days of hourly data
output_seq_len = 24 # 1 day of hourly data
n_features = len(features)

In [32]:
# Define the LSTM model
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(input_seq_len, n_features)))
model.add(Dense(output_seq_len * n_features))
model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [61]:
# Train the LSTM model
X_train = np.array([train_data.iloc[i:i+input_seq_len, :].values for i in range(len(train_data) - input_seq_len - output_seq_len)])
y_train = np.array([train_data.iloc[i+input_seq_len:i+input_seq_len+output_seq_len, :].values.flatten() for i in range(len(train_data) - input_seq_len - output_seq_len)])
model.fit(X_train, y_train, epochs=5, batch_size=128, verbose=1)

Epoch 1/5
109/109 [==============================] - 46s 410ms/step - loss: nan
Epoch 2/5
109/109 [==============================] - 36s 333ms/step - loss: nan
Epoch 3/5
109/109 [==============================] - 36s 329ms/step - loss: nan
Epoch 4/5
109/109 [==============================] - 34s 310ms/step - loss: nan
Epoch 5/5
109/109 [==============================] - 37s 342ms/step - loss: nan


In [62]:
# Evaluate the LSTM model on the test data
X_test = np.array([test_data.iloc[i:i+input_seq_len, :].values for i in range(len(test_data) - input_seq_len - output_seq_len)])
y_test = np.array([test_data.iloc[i+input_seq_len:i+input_seq_len+output_seq_len, :].values.flatten() for i in range(len(test_data) - input_seq_len - output_seq_len)])
# print(len(X_test))
# print(len(train_std))
y_test = (y_test - train_mean) / train_std
train_mean_broadcast = np.broadcast_to(train_mean, X_test.shape)
train_std_broadcast = np.broadcast_to(train_std, X_test.shape)

X_test = (X_test - train_mean_broadcast) / train_std_broadcast
# X_test = (X_test - train_mean) / train_std
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score)

ValueError: operands could not be broadcast together with shapes (3321,264) (11,) 

In [5]:
l=[]
l.pop()
l

IndexError: pop from empty list